In [1]:
## Preloads
using Statistics
using FFTW
using Plots
using BenchmarkTools
using Profile
using LinearAlgebra
using Measures
using HDF5
push!(LOAD_PATH, pwd())
using DHC_2DUtils
using MLDatasets
using Images
theme(:juno)
using DSP
using Interpolations
using Distributed
using ProgressMeter
using ImageTransformations

  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:162
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:162
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:166
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:166
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:176
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:179
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:181
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:184
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:186
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:189
  likely near /n/home12/saydjari/.julia/packages/Plots/uCh2y/src/backends/hdf5.jl:203
  likely near /n/home12/saydjari/.julia/packages/Plots

In [2]:
test_img = rand(64,64);

In [3]:
imresize(test_img,Dims((128,128)),Lanczos4OpenCV());

In [4]:
imrotate(test_img,0.3,axes(test_img), Lanczos4OpenCV());

In [14]:
workers()

1-element Array{Int64,1}:
 1

In [15]:
addprocs(30)

30-element Array{Int64,1}:
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
  ⋮
 20
 21
 22
 23
 24
 25
 26
 27
 28
 29
 30
 31

In [7]:
M = 100
angle_array = []
for i = 2π/M:2π/M:2π
    append!(angle_array,i)
end

M = 100
train_angles = [2π/M,π/3,2π/3,3π/3,4π/3,5π/3];

In [8]:
train_x, train_y = MNIST.traindata()
test_x, test_y   = MNIST.testdata()

lst_train = Array{Any}(undef, 0)
for i = 1:60000
    push!(lst_train,train_x[:,:,i])
end


lst_test = Array{Any}(undef, 0)
for i = 1:10000
    push!(lst_test,test_x[:,:,i])
end

In [9]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    using ImageTransformations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=256,wd=2,pc=1)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,128,128)
        impad[78:-1:51,51:78] = im'
        imbig = imresize(impad,(256,256), Lanczos4OpenCV())
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Lanczos4OpenCV())
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash)
        return WST
    end
    
end
 
mnist_DHC_out = @showprogress pmap(mnist_DHC, Iterators.product(angle_array,lst_test))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_test_ang_LanRotResize.h5", "data", mnist_DHC_out, deflate=3)


Progress: 100%|█████████████████████████████████████████| Time: 4:31:38


MethodError: MethodError: no method matching datatype(::Array{Any,1})
Closest candidates are:
  datatype(!Matched::HDF5.Attribute) at /n/home12/saydjari/.julia/packages/HDF5/d0V7K/src/HDF5.jl:1017
  datatype(!Matched::HDF5.Dataset) at /n/home12/saydjari/.julia/packages/HDF5/d0V7K/src/HDF5.jl:1015
  datatype(!Matched::Union{Bool, Float32, Float64, Int16, Int32, Int64, Int8, UInt16, UInt32, UInt64, UInt8, HDF5.Reference}) at /n/home12/saydjari/.julia/packages/HDF5/d0V7K/src/HDF5.jl:1020
  ...

In [13]:
h5write("mnist_DHC_test_ang_LanRotResize.h5", "angles", convert(Array{Float64},angle_array), deflate=3)

In [16]:
@everywhere begin
    using Statistics
    using BenchmarkTools
    using LinearAlgebra
    using Distributed
    using ProgressMeter
    push!(LOAD_PATH, pwd())
    using DHC_2DUtils
    using FFTW
    using MLDatasets
    using Images
    using Interpolations
    using ImageTransformations
    
    # filter bank
    filter_hash = fink_filter_hash(1,8,nx=256,wd=2,pc=1)
    
    function mnist_pad(im; θ=0.0)
        impad = zeros(Float64,128,128)
        impad[78:-1:51,51:78] = im'
        imbig = imresize(impad,(256,256), Lanczos4OpenCV())
        if θ != 0.0
            imrot = imrotate(imbig, θ, axes(imbig), Lanczos4OpenCV())
            imrot[findall(imrot .!= imrot)] .= 0.0
            return imrot
        end
        return imbig
    end
    
    function mnist_DHC(params)
        θ, x = params
        image    = mnist_pad(x[:,:], θ=θ)
        WST = DHC_compute(image, filter_hash)
        return WST
    end
    
end

In [ ]:
mnist_DHC_out = @showprogress pmap(mnist_DHC, Iterators.product(train_angles,lst_train))
mnist_DHC_out = hcat(mnist_DHC_out...)

h5write("mnist_DHC_train_ang_LanRotResize.h5", "data", mnist_DHC_out, deflate=3)
h5write("mnist_DHC_train_ang_LanRotResize.h5", "angles", convert(Array{Float64},train_angles), deflate=3)

Progress:  42%|█████████████████▏                       |  ETA: 0:42:16